In [5]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

ARQ = '../data/raw/imdb_top_250.csv'

df_filmes = pd.read_csv(ARQ,
                        sep = ';',
                        decimal = ',',
                        encoding = 'latin1'
)

df_filmes.head()


,Título,Ano,Nota,Sinopse,Diretor,Elenco Principal
0,Um Sonho de Liberdade,1994,9.3,Dois homens presos se reúnem ao longo de vário...,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton"
1,O Poderoso Chefão,1972,9.2,O patriarca idoso de uma dinastia do crime org...,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan"
2,Batman: O Cavaleiro das Trevas,2008,9.1,Agora com a ajuda do tenente Jim Gordon e do p...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart"
3,O Poderoso Chefão: Parte II,1974,9.0,O início da vida e da carreira de Vito Corleon...,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall"
4,12 Homens e uma Sentença,1957,9.0,O julgamento de um assassinato em Nova Iorque ...,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam"


In [3]:
df_filmes.shape

(250, 6)

In [4]:
df_filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Título            250 non-null    object 
 1   Ano               250 non-null    int64  
 2   Nota              250 non-null    float64
 3   Sinopse           250 non-null    object 
 4   Diretor           250 non-null    object 
 5   Elenco Principal  250 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 11.8+ KB


In [ ]:
# Normalizando nomes das colunas (acentos e espaços)
def normalizar_colunas(df):
    df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # Normalize separa acento das letras, encode remove acento, decode volta a ser string
    df.columns = df.columns.str.replace(' ', '_').str.lower() # Substitui espaços por _ e deixa tudo minúsculo
    return df

df_filmes = normalizar_colunas(df_filmes)
print("Colunas após normalização:")
print(df_filmes.columns.tolist())

Colunas após normalização:
['titulo', 'ano', 'nota', 'sinopse', 'diretor', 'elenco_principal']


In [ ]:
# Verificando e removendo duplicatas
print(f"\nShape antes de remover duplicatas: {df_filmes.shape}")
df_filmes = df_filmes.drop_duplicates()
print(f"Shape após remover duplicatas: {df_filmes.shape}")


Shape antes de remover duplicatas: (250, 6)
Shape após remover duplicatas: (250, 6)


In [9]:
# Verifcando valores nulos
print("\nValores nulos por coluna:")
print(df_filmes.isnull().sum())


Valores nulos por coluna:
titulo              0
ano                 0
nota                0
sinopse             0
diretor             0
elenco_principal    0
dtype: int64


In [ ]:
# Lipando dados textuais
def limpar_texto(coluna):
    if coluna.dtype == 'object':
        return coluna.str.strip().str.replace(r'\s+', ' ', regex=True) # Remove possíveis espaços extras
    return coluna

df_filmes = df_filmes.apply(limpar_texto)

In [20]:
# Normalizando nomes próprios (Diretor e Elenco)
def normalizar_nomes_proprios(texto):
    if pd.isna(texto):
        return texto
    
    # Lista de conectores/preposições que devem ficar em minúsculas
    conectores = {"de", "da", "do", "dos", "das", "e"}
    
    nomes = texto.split(", ")
    nomes_normalizados = []
    
    for nome in nomes:
        palavras = nome.split(" ")
        palavras_normalizadas = []
        for i, palavra in enumerate(palavras):
            if palavra.lower() in conectores and i != 0:  
                # Mantém conector em minúscula, exceto se for a 1ª palavra
                palavras_normalizadas.append(palavra.lower())
            else:
                palavras_normalizadas.append(palavra.capitalize())
        nomes_normalizados.append(" ".join(palavras_normalizadas))
    
    return ", ".join(nomes_normalizados)

# Aplicando
df_filmes['diretor'] = df_filmes['diretor'].apply(normalizar_nomes_proprios)
df_filmes['elenco_principal'] = df_filmes['elenco_principal'].apply(normalizar_nomes_proprios)


In [21]:
print(df_filmes['elenco_principal'].head())

0        Tim Robbins, Morgan Freeman, Bob Gunton
1           Marlon Brando, Al Pacino, James Caan
2    Christian Bale, Heath Ledger, Aaron Eckhart
3       Al Pacino, Robert de Niro, Robert Duvall
4        Henry Fonda, Lee J. Cobb, Martin Balsam
Name: elenco_principal, dtype: object


In [11]:
print(f"\nRange de notas: {df_filmes['nota'].min()} - {df_filmes['nota'].max()}")


Range de notas: 8.0 - 9.3


In [23]:
caminho_limpo = '../data/processed/imdb_top_250_limpo.csv'
df_filmes.to_csv(caminho_limpo, index=False,
                                  sep = ';',
                                  decimal = ',',
                                  encoding = 'latin1'
)
print(f"\nBase salva em: {caminho_limpo}")


Base salva em: ../data/processed/imdb_top_250_limpo.csv
